In [6]:
from model import load_lstm_model
from main import load_and_preprocess_data
from predict import predict_next_transaction
import pandas as pd

df, encoders = load_and_preprocess_data()

df.head(10)

cc_num = df['cc_num'].iloc[0]

user_transactions = df[df['cc_num'] == cc_num]

sequence_length = 10

lstm_model, scaler, encoders = load_lstm_model()

next_transaction = predict_next_transaction(
    lstm_model, user_transactions, sequence_length, scaler, df.columns, encoders
)

next_transaction

Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%


Loading model from lstm_transaction_model.h5...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


,amt,hour,day,month,dayofweek,merchant_encoded,category_encoded,lat,long,merch_lat,merch_long,is_fraud,merchant,category
0,65.624756,13.099962,20.686098,6.10028,5.258587,332.718079,6.363268,42.276993,-109.734459,42.313931,-109.741455,0,"fraud_Koss, Hansen and Lueilwitz",home
